This Jupyter notebook makes use of the Library of Simulations and Base funcs to automatically run a simulation for a number of charges and a set number of simulations. Making use of a random walk simulation to calculate the temperature reduction needed, and generating lots of initial coordinates to calculate the standard deviation fo energies to get the correct initial temperature value.

In [9]:
import matplotlib.pyplot as plt
import numpy as np
# imports my own created library
import ChargedDisk.Simulation as sim
from ChargedDisk.BaseFuncs import Mult_pass as MP
from ChargedDisk.BaseFuncs import Single_pass as SP
import math
import time
# so it updates in main
# the random walk function, very simple
def walk(walkers: int, steps = 100, dims = 3, delta = 1):
    walks = 2*np.random.rand(walkers,steps, dims)-1
    walks *= delta
    dist = np.cumsum(walks, axis = 1)
    return dist


def total_sim(re_runs, num_charges):
    print('Running simulation for {} charges'.format(num_charges))
    print('Running {} simulations'.format(re_runs))

    disk_radius = 100 # choice of disk radius, arbitrarily chose 100
    P = 0.9 # the initial probability of accepting a higher energy change
    delta = 3 # the delta or the multiplier for each random movement of a charge
    charge_movements = 100 # the number of charge movements per temperature reduction
    precision = 4
    walkers = np.power(10,precision) # the number of walkers to generate to extract the data from
    dims = 2 # the dimension of the random walk simulation
    total_steps = 10*num_charges # the initial total steps needed so that charges will move across the radius of the disk
    steps = int(total_steps/num_charges) # get the step needed per charge
    distances = walk(walkers, steps, dims, delta = delta) # return the random walk data
    distances = np.linalg.norm(distances,axis=-1) # calculate the distances reached by each walker
    reached = np.unique(np.where(distances > disk_radius)[0]) # find the index where each walker has reached the radius of the disk

    while len(reached)/walkers < 0.95: # while the probability of each walker reaching the radius of the disk
        steps = int(total_steps/num_charges) # calculate the steps
        distances = walk(walkers, steps, dims, delta = delta) # run the random walk sim
        distances = np.linalg.norm(distances,axis=-1) # calculate the distance
        radius = 100
        reached = np.unique(np.where(distances > radius)[0]) # calcualte how many walkers reached the radius
        total_steps *= 1.1 # increase the total steps by 10%
    print('random walk data', len(reached), len(reached)/walkers, int(total_steps)) # print the data

    accuracy = 4
    sample_size = np.power(10,accuracy)
    charges = MP.gen_coords(sample_size, disk_radius, num_charges)
    energies = np.squeeze(np.round(MP.calc_energy(charges, charge=1), accuracy))
    sigma = np.std(energies) # calculate the standard deviation

    temp = - 3 / math.log(P)
    temp *= sigma # calculate the initial temperature from the standard deviation 
    print('initial temperature', temp)
    t0 = temp
    precision = 6 # chooses the precision of the temperature reduction
    log = -np.log(t0*np.log(2))
    R = np.arange(0.95,1,np.power(float(np.power(10,precision)),-1))
    R = np.delete(R,-1)
    calc = np.log(R)
    calc = np.power(calc,-1)
    vals = np.stack((log*calc, R), axis = 1) # calculates the number of iterations for the probability to reach 50% chance of accepting a higher energy state for different reductions in temperature
    lowest_iters = vals[vals[:,0] > int(total_steps/charge_movements)] # finds the reduction value that gives the number of iterations needed (calculated from random walk)

    reduct = round(lowest_iters[0,1],precision+1) # rounds the reduciton of temperature value
    print('total steps needed and reduction value:',int(total_steps), round(reduct,precision+1))
    t0 = time.perf_counter()
    # runs the simuilation with the calcualted intial conditions
    coords, energies, iterations = sim.Run_Sim_MP(\
        disk_radius, num_charges, delta = delta, temp = temp, charge_movements = charge_movements,runs = re_runs, decimals = 8, reduction=reduct)
    print('time taken {} minutes {} seconds'.format(*divmod(time.perf_counter()-t0,60)))
    # finds the lowest energy state from all the simulations run
    lowest_state = np.where(energies[:,-1] == np.min(energies[:,-1]))
    # returns the lowest state charge coordinates and the energy
    return np.squeeze(coords[lowest_state]), np.squeeze(energies[lowest_state]), temp, sigma, reduct

sims_dict = {} # creates a dictionary so that simulation data can be stored
sim_num = 200
for i in range(20, 40):
    # creates a dictionary for each number of charges and then saves the simulation data to a file
    sims_dict['{}'.format(i)] = {}
    sims_dict['{}'.format(i)]['coords'],\
          sims_dict['{}'.format(i)]['energies'],\
              sims_dict['{}'.format(i)]['inital temperature'],\
                  sims_dict['{}'.format(i)]['sigma'],\
                     sims_dict['{}'.format(i)]['temperature reduction'] = total_sim(sim_num, i)
    sims_dict['{}'.format(i)]['simulations'] = sim_num
    try:
        with np.load('models/simulation {} charges.npz'.format(i)) as data:
            data = dict(data)
            sims_dict['{}'.format(i)]['simulations'] += data['simulations']
            if sims_dict['{}'.format(i)]['energies'][-1] < data['energies'][-1]:
                np.savez('models/simulation {} charges.npz'.format(i), **sims_dict['{}'.format(i)])
            else:
                data['simulations'] += sim_num
                np.savez('models/simulation {} charges.npz'.format(i), **data)
    except:
        print('no sims run for this number of charges')
        np.savez('models/simulation {} charges.npz'.format(i), **sims_dict['{}'.format(i)])

Running simulation for 20 charges
Running 200 simulations


KeyboardInterrupt: 